# Trade Bot Analysis

## Background 

Ripple's XRP is a top 5 market cap cryptocurrency traded on multiple markets. The price of XRP is highly volatile and commonly fluctuates multiple percent per day with the occasional bull/bear runs. 

One of the more popular exchanges for trading XRP is Bitstamp. Bitstamp has a open [api](https://www.bitstamp.net/api/) which allows the user to extract real time market data and information connected to the users account. This allows the user create trades through the api, giving the possibilty to set up trading automation in form of a bot which implements a trading strategy.  

# Trading Strategy

Using the daily fluctuation in price a naive strategy could be:

<ol>
  <li>Buy at a price X</li>
  <li>Sell at price Y, where price why is a big enough increase in price to gain some profit while covering fee</li>
  <li>Repeat the process when price is X using the intial value or inluding profits to purchase</li>
</ol>

### Strategy Drawbacks

* The price might not go up enough to sell
* The price might not go down again after selling.

## Mathematical  Formulation and Analysis

#### Nomenclature
$P$: Profit  [£] <br>
$V_0$: Value of initial investment [£]<br>
$i$: Interest [%] <br>
$f_b$: Buy Fee  [%] <br>
$f_s$: Sell Fee  [%] <br>
$n$: Number of succesful cycles <br>
$c$: Increased value [%] <br>

#### Formula for Profit Excluding Reinvestint Profits
The profit per succesful cycle when reinvesting **excluding** profits is given by,

$$P = V_0(1+i)(1-f_b)(1-f_s) - V_0 = V_0((1+i)(1-f_b)(1-f_s) - 1)$$
$$c = (1+i)(1-f_b)(1-f_s) $$
$$P = V_0(c-1)$$

The potential total proft is given by,

$$ P_{tot} = \sum_{i=0}^n P_i = nP$$

#### Formula for Profit Including Reinvestint Profits
The profit per succesful cycle when reinvesting **including** profits is given by,

$$V_0 = V_0$$
$$V_1 = V_0c$$
$$V_2 = V_1c=V_0c²$$
$$V_3 = V_2c=V_0c³$$
$$\vdots$$
$$V_n = V_0c^n$$.
$$P_n = V_n - V_0 =  V_0c^n - V_0 = V_0 (c^n - 1)$$

#### Required Interest Rate
The minimum interest required to break even $P=0$ even is, 

$$c = (1+i)(1-f_b)(1-f_s)  = 1  \rightarrow i = \frac{1}{(1-f_b)(1-f_s) } - 1$$

On bitstamp $f_b = f_s = 0.5\%$ which gives

$$P(0.005, 0.005) = 0 \rightarrow  i = 0.0100755031 \rightarrow 1.00755031\% $$

as the break even interest rate. 

On Kraken $f_b = 0.26\%$ and $f_s = 0.26\%$ with taker fees which gives,

$$P(0.0026, 0.0026) = 0 \rightarrow  i =  0.0052203505 \rightarrow 0.52203505\% $$.

With maker fees $f_b = 0.16\%$ and $f_s = 0.16\%$,

$$P(0.0016, 0.0016) = 0 \rightarrow  i =  0.0032076964 \rightarrow 0.32076964  \% $$

The change in profit based on changes in interest can be described by the derivative,

$$\frac{dP}{di} = V_0(1-f_b)(1-f_s)$$

the increase in profit based on changes in interest is constant. 

### Visualisation

In [16]:
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go

# Constants
initial_value = 1000
interest = 0.007
succesful_cycles = np.arange(50)

In [17]:
# Bitstamp 0.5% fees
fee_buy = 0.005
fee_sell = 0.005
value_increase = (1 + interest) * (1- fee_sell) * (1 - fee_buy)
bit_stamp_profit_reinvesting_excluding_profit = [n * (value_increase - 1) * 100 for n in succesful_cycles]
bit_stamp_profit_reinvesting_including_profit = [(value_increase**n - 1) * 100 for n in succesful_cycles]

In [18]:
# Kraken 0.26% fees
fee_buy = 0.0026
fee_sell = 0.0026
value_increase = (1 + interest) * (1- fee_sell) * (1 - fee_buy)

kraken_profit_reinvesting_excluding_profit_26 = [n * (value_increase - 1) * 100 for n in succesful_cycles]
kraken_profit_reinvesting_including_profit_26 = [(value_increase**n - 1) * 100 for n in succesful_cycles]


In [19]:
# Kraken 0.16% fees
fee_buy = 0.0016
fee_sell = 0.0016
value_increase = (1 + interest) * (1- fee_sell) * (1 - fee_buy)

kraken_profit_reinvesting_excluding_profit_16 = [n * (value_increase - 1) * 100 for n in succesful_cycles]
kraken_profit_reinvesting_including_profit_16 = [(value_increase**n - 1) * 100 for n in succesful_cycles]


In [20]:
# Edit the layout
fig = go.Figure()
fig.add_trace(go.Scatter(x=succesful_cycles, y=bit_stamp_profit_reinvesting_excluding_profit,
                    mode='lines',
                    name='Bitstamp: Excluding Profit'))
fig.add_trace(go.Scatter(x=succesful_cycles, y=bit_stamp_profit_reinvesting_including_profit,
                    mode='lines',
                    name='Bitstamp: Including Profit'))
fig.add_trace(go.Scatter(x=succesful_cycles, y=kraken_profit_reinvesting_excluding_profit_26,
                    mode='lines',
                    name='Kraken 26: Excluding Profit'))
fig.add_trace(go.Scatter(x=succesful_cycles, y=kraken_profit_reinvesting_including_profit_26,
                    mode='lines',
                    name='Kraken 26: Including Profit'))
fig.add_trace(go.Scatter(x=succesful_cycles, y=kraken_profit_reinvesting_excluding_profit_16,
                    mode='lines',
                    name='Kraken 16: Excluding Profit'))
fig.add_trace(go.Scatter(x=succesful_cycles, y=kraken_profit_reinvesting_including_profit_16,
                    mode='lines',
                    name='Kraken 16: Including Profit'))
fig.update_layout(title=f'Profit Per Succesful cycle with interest={interest*100}%',
                   xaxis_title='Succesfull Cycle',
                   yaxis_title='Profit [%]')
fig.show()

## Conclusion 
Kraken is alot cheaper and more profitable. Also note that kraken has maker fees which are 0.16% instead of 26\% as takers fees which can be taken into consideration in the strategy. 

## Algorithmic Implementation

<ol>
  <li>Buy in at previous day low price (assuming there is not bull/bear run)</li>
  <li>Check ask prices on a t sec interval</li>
  <li>If ask price is the required percent increase then sell</li>
  <li>Check ask prices on a t sec interval</li>
  <li>If ask price is = original buy price then buy </li>
  <li>Continue and repeat</li>
</ol>
